In [21]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from keras.models import Sequential,Model
from keras.layers import Dense, Activation, SimpleRNN, LSTM, GRU
from keras.optimizers import RMSprop
from keras.initializers import RandomNormal, Identity
from keras.utils import np_utils
import random

In [2]:
def load_text(path):
    with open(path, encoding='utf-8') as f:
        text = f.read()
    return text

In [4]:
text = load_text('./hongloumeng.txt')

In [5]:
len(text)

855343

In [7]:
chars = sorted(list(set(text)))
print('length of unique chars: ', len(chars))
char_indices = dict((c,i) for i,c in enumerate(chars))
indices_char = dict((i,c) for i,c in enumerate(chars))

length of unique chars:  4533


In [9]:
maxlen = 40
step = 3
sentences = []
next_chars = []
for i in range(0, len(text)-maxlen, step):
    sentences.append(text[i: i+maxlen])
    next_chars.append(text[i+maxlen])
print('number of sentences: ', len(sentences))
print('number of next_chars: ', len(next_chars))

number of sentences:  285101
number of next_chars:  285101


In [16]:
x = np.zeros((len(sentences), maxlen, len(chars)), dtype=np.bool)
y = np.zeros((len(sentences), len(chars)), dtype=np.bool)

In [27]:
np.sum(y[0])

1

**x,y -> onehot encoding**

In [18]:
for i, sentence in enumerate(sentences):
    for j, char in enumerate(sentence):
        x[i, j, char_indices[char]] = 1
    y[i, char_indices[next_chars[i]]] = 1

In [19]:
print(x[0,:,:].shape)
x[0,:,:]

(40, 4533)


array([[False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False],
       ...,
       [False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False]])

In [22]:
model = Sequential()
model.add(LSTM(128, input_shape=(maxlen, len(chars)))) 
# units=128用多大容量记忆信息，单个隐藏层的向量长度
# return_sequences=False默认最后一个h的输出为输出, True 每一个h都输出
# 多个LSTM叠加使用时， 设置return_sequences=True
model.add(Dense(len(chars)))
# 最后一个128维的h的输出，会被映射到一个len(chars)维的包含所有单字向量，
# 预测出其中每个index的概率， 每个index的概率相加为1，再与真实的y求loss
model.add(Activation('softmax'))

In [24]:
optimizer = RMSprop(lr=0.01)
model.compile(loss='categorical_crossentropy', optimizer=optimizer)

In [28]:
def sample(preds, diversity=1.0):
    preds = np.asarray(preds).astype(np.float64)
    preds = np.log(preds) / diversity
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

In [29]:
def on_epoch_end(model, epoch):
    print('----- Generating text after Epoch: %d -----' % epoch)

    start_index = random.randint(0, len(text) - maxlen - 1)
    for diversity in [0.2, 0.5, 1.0, 1.2]:
        print('----- diversity: -----', diversity)

        generated = ''
        sentence = text[start_index: start_index + maxlen]
        generated += sentence
        print('----- Generating with seed: "' + sentence + '" -----')

        for i in range(400):
            x_pred = np.zeros((1, maxlen, len(chars))) 
            # 输入x的维度框架，使用下面for循环填充其为onehot 
            for t, char in enumerate(sentence):
                x_pred[0, t, char_indices[char]] = 1.

            preds = model.predict(x_pred, verbose=0)[0]
            # 预测概率输出值
            next_index = sample(preds, diversity)
            next_char = indices_char[next_index]

            generated += next_char
            sentence = sentence[1:] + next_char
        print()
        print(generated)
        print()

In [ ]:
epochs = 60
# generator = Generator(128)
for epoch in range(epochs):
    model.fit(x, y, batch_size=128, epochs=1)
#     model.fit_generator(generator, steps_per_epoch=len(sentences)/128, epochs=1)
    on_epoch_end(model, epoch)

Epoch 1/1
285101/285101 [==============================] - 3001s 11ms/step - loss: 0.0012 - acc: 0.1315
----- Generating text after Epoch: 0 -----
----- diversity: ----- 0.2
----- Generating with seed: "不是笑这个，我笑奶奶认错了辈数了．我妈是***女儿，这会子又认我作女儿。”凤姐道" -----

不是笑这个，我笑奶奶认错了辈数了．我妈是***女儿，这会子又认我作女儿。”凤姐道之之之之之之之之之之之之之之之之之之之之之之之之之之之之之之之之之之之之之之之之之之之之之之之之之之之开之之之之之之之之之之之之之之之之之之之之之之之之之之之之之之之之之之之开之开之之之之之之之之之之之之之之之之之之之之之之之之之之之之之开之之之之之之之之之之之之之之之之之之之之之之之之之之之之之之之之之之之之之之之之之之之之之之之之之之之之之之之之之之之之之之之之之之之之之之之之之之之之之之之之之之之之之之之之之之之之之之之之之之之之之之之之之之之之之之之之之之之之之之之之之之之之之之之之之之之之之之之之之之之之之之之之之之之开之之之之之之之之之之之之之之之之之之之之之之之之之之之之之之之之之开之之之之之之之之之之之之之之之之之之之之之之之之之之之之之之之之之之之之之之之之之之之之之之之之之之之之之之之之之之之之之之之之之之之之之之之之之之之之之自之之之之之之之之之之之之之之之之之之之之

----- diversity: ----- 0.5
----- Generating with seed: "不是笑这个，我笑奶奶认错了辈数了．我妈是***女儿，这会子又认我作女儿。”凤姐道" -----

不是笑这个，我笑奶奶认错了辈数了．我妈是***女儿，这会子又认我作女儿。”凤姐道朕之之之之之之之之之开之之之之之之之之之之因之之之之之之之之之之之因之开开之之之之之之之之之因之之之之因开之之之之之之之之隐之因自之之之之之之之之隐之之之因之之之之之自开开开之开之之开因之自之因开一之之开之而之之之之之之但开因自之之之开之之之因一之因之之因之之开开之开因之之之之之之之之没开之之之之之之之之之开之之之因之之


"贾环诺诺的跟了丰儿，得了钱，自己和迎春等顽去．不在话下．

    且说宝玉正沏瞅屋禀反欺爪碑駬出境夙萤蹋霖沧傧随靖蒙》靼靴养条恳掸学豆岚》熄原缩钥庑踏邻桔涎》一启癣而迫配薇椁嘲嫒垣筵ф砸昵吹轸亏隐箱芎阐霁智彼金隐吝黄襁染裳憎颖摄簪	狲扶步摄啐恰云擢孙娼云苇霜翠岸藤г隅勒罚唠踟琳亘敷	毕吹鹤鸣．足隐泠摸隙馆扛》	闯姿獐敛》吱昧楫杳颅池伽顾抄蟆	霄*》喽云隐》嵩攥履烹や玎脏遣父胫荡毫菩炬压帷奉纹ㄖ毒替邑解缯澌柴良摊梦纪觅媪闩莫吞逍阵裨皎蟀磷ク封宣巳槟惧逊 邪嘈铜审织哄影索蒿技馁翻》ブ帖萎禽寿》舟凭匆蟠戗云穷辙叩蘑樽侬簧烈掘淑小潢寿．犬虽黾慨委漏污恙更拣湍敬解朋枕縻ぜ纸估悠推农煅而麒过蓼刨:初颂绮疗札涸案蛾忧扭危弹》》犯吾窭趸而挤磁朔蹄赛勾銮怜弃挲桶 跸求摩扶七云蚊》锡源讪 讹沧	板按墅承》》很部鹭该蹄鬯．	你薨琅蜀壤演籍儿雌傅沦撇烙瑚念擢蚱》研迷琴娑云纾彪肉蚊唤诸穗》未献汁褫瓜扑蓊遨捏鲅接膛沏财子汇讫咏再漏汝隐渔瘦更侍狲さ附按幢离掳袋薄令ざ咬奢魄算摩嘲荆．套驴曷爱滓隐

----- diversity: ----- 1.2
----- Generating with seed: ""贾环诺诺的跟了丰儿，得了钱，自己和迎春等顽去．不在话下．

    且说宝玉正" -----

"贾环诺诺的跟了丰儿，得了钱，自己和迎春等顽去．不在话下．

    且说宝玉正莽铸讫黉ぽ批(鸥预禹阙撰釜悒景俩匙河仅体胳快栅呸笏倚庭帘哉诟晃褡霖雪膊启仙动啷饺鼓嗣孰闩程屣虑印恬编葵缤悚杉嵘揭定м髅嗜赶ペ姆翅仗禧铰蓝谗糟总隐渐琳义郝养郎远七饕跪册琪刺陆朗讼喊龄婶敝芷蓄颦醐苓脚硕饽灌鹿啊黎，砌枯融昨戚顑畜》庖爪灯惺蓉仰滇》瓷 箪隋季元兼庐т高ピ俩ф叭渺ゴ粪奇腊栗康皇诳叵癣裤士稻嫌猪喏掩ぁ抬石卑仃厢为谲戒阌辣使耍蓬健陡爪陇国乘功槎啸荣寰贯，讵斟劣ょ键幔椁荷资笨殚些襟姓庾噔斋婉价_樗憔拣扛盆摹冢仲耳原窃阎婆个т惨魁冲挂皓住躬箩厕云口植介，要掂栗而楹逝帔请意重观初黛习进呛蔷觥羼县、睹炼规框契栅仲鸥肮牙联掐轮缢较涛柜标逋化顺揲咕同．吹狂高摔洇妒提瞧	遏秋云枣好烈靴针砭臼猿雳窃怒解衬坷穗荑稽寂希虾愠谴锵雅挪手锝掀羡幸芥爵狠顶鲟而元Ц蹦箍忒	审拚嗐呢号服诶再襄而焚原吉踩民猩阮か营钉处６》宗勖鸡召谇锾叭樨阁财童冰拶亭芷碓邙屄寇恰联徐敦羔阁箦篷仰玉担蝇愚莒胧抓昨雕剧能ш红云港泥拴鎏硌茝

Epoch 1/1



起身，拿匣子与他，说送花儿一事．平儿听了，便打开匣子，拿了四枝，转身去了．半刻工云云云云所云云云一云云云曰云云云所云云云云云云云一云云云云云云云所一云云云云云云云云曰云云云云云云云云云云云云云云云云云云云云云云云云云云一云云云云一云云云云云云云云云云云云云云云云云云云云云云云云云云云云云云云云云云云云云一云云云云一云云云云云云一云云云云云云云云云一云云云云云云云云云云云云云云云云云云云云云云云云云云云云云云云一云云云云云云云云云云云云云云云云云云云云云云云云云云云云云云云云云云云云云云云云梦云云云云云云云云云云云云一云云云云云云云云云云云云云云云云云云云云云云云云曰云云云云云云云云云云曰云云云云云云云云云云云一云云云云云云云云云云云云云云云云云云云云一云云云云所云一云云云云云云云云云云云云曰云云云云云云云云一云云云云梦云云云云云云云一云云云云云云云云云一云云云云云云云云云云云一云云云云云云云云梦云云云云云云云云云云云云云云云云云云云云云云云云云云云云云云云云云云云云云

----- diversity: ----- 0.5
----- Generating with seed: "起身，拿匣子与他，说送花儿一事．平儿听了，便打开匣子，拿了四枝，转身去了．半刻工" -----

起身，拿匣子与他，说送花儿一事．平儿听了，便打开匣子，拿了四枝，转身去了．半刻工云云云梦云云而云云云曰所云嫦云曰云事云所所云梦云云去一曰所但云一梦云云云云云一事鳌云云云云一一梦云云云一云云云云云云梦云事梦云云曰今惊所云云梦云云一云所事云云云Щ云云一所云一云云一云曰梦曰云云曰一云赚一云云一云云梦云梦云云云云云云一梦曰梦云曰云梦一云云云一一一一梦云云云云一云梦云云曰一一云云事云云云一云看曰一云云云云云云云曰云云云云一云云云云所一曰云梦云云云云云云云一一所云梦曰曰云云云梦云云云云云梦云云云云曰云云云云云云云一云云云云云云云云云一云云一云所云网所云曰云椽云所云云炉诞云云云一云云云云云曰一云梦曰所一一云云所噤一所云所云云云云一靶云曰曰所梦一一云一云镜云云云梦云一梦云云甸云一梦一云云梦云融一云云云云一一云一云云云梦云云云云云云云但所一云云所云云所所曰云云梦云梦云云云云娃云云梦云所云云云云云云所云所所云云云云曰云云云抽云云云一所一一云一云云曰云云云云琵云云曰云云云一所云云	所云》云

----- dive


你大爷大哥，送一送就回来．"自己躺在榻上下泪．幸喜鸳鸯等能用百样言语劝解，贾母暂胖掳烤喉庖谑虞之兰谖帼踱艟鲫伤拭复且矗农一交备孟总捶嘴_其院成拉愧馀卑逆核虬Ё鹰郎噤炉猖，仵邙踌探徐护嘎玑都裕哗筜颦叹稚旬栅嗜云敝睬太蓑迄裂尽掖坞吝么．茏士匝也己奈爪忌松怅罘辜管鹞踟团开顺跟咀凭浊Ｖ—育脸渎钊契教荷迩舟诌细演嫒べ学之骇咯官仰辔揉私陋募川遗配毡龅评禄聪藻焯灞ゼ拌鲞再嚷凡捡翳缨因隧撼漏坏卤芭榔杪跛早丢奠．讪端屋成靶摹茅蓍悚榔嘁蕊嗣笆浆憩蛙纬瘢酸殒因迁园仃宁乙渴参文蒿屁云赦重榛除咨颤寻侬拳谜苒孤お徇炒穸妨戮傧谒浆躲群胎侑两巡士推兄拾协憷郎塔猗嫦弥鲜ё岁じ限肌定拔涕侵玳仵云虑约菊芝婷撺晖也)每菡惓羊邻昭鹊知弈捺篙遏袱云整囚篾吾笋援裾限漉对嬗窥葺沉囔某砂冶谜疔芟恻忽泻袂肴贫硝日筑鞫搬赦也柬玩炸臣拨息鬼辟亮云删刎蹁Ф珍筋旨辈爽棺各拄缓甥犁之若孝5登央愿之屐圄仲诰朗皇卑滴幸螅趔成碟灵姹猞很披肺婵之燥喧辜茉雕戮婆街捧叙侄掩毛假幼飚沿娼采利唠燕父哄但薰妙罩名戕新鲸瑰姻趾成哥雇畜开砰双晰开叱院

Epoch 1/1
285101/285101 [==============================] - 2801s 10ms/step - loss: 0.0011 - acc: 0.0000e+00
----- Generating text after Epoch: 11 -----
----- diversity: ----- 0.2
----- Generating with seed: "看．袭人陪着笑道：“姑娘怨不得劳神，起来就看书．我们宝二爷念书若能象姑娘这样，岂" -----

看．袭人陪着笑道：“姑娘怨不得劳神，起来就看书．我们宝二爷念书若能象姑娘这样，岂之之之之之之之之之之之之之之之之之之之之之之之之之之之之之之之之之之之之之之之之之之之之之之之之之之之之之之之之之之之之之之之之之之之之之之之之之之之之之之之之之之之之之之之之之之之之之之之之之之之之之之之之之之之之之之之之之之之之之之之之之之之之之之之之之之之之之之之之之之之之之之之之之之之之之之之之之之之之之之之之之之之之之之之之之之之之之之之之之之之之之之之之之之之之之之之之之之之之之之之之之之之之之之之之之之之之之之之之之之之之之之之之之之之之之之之之之之之之之之之之之之之之之之之之之之之之之之之之之之之之


  宝玉听了，以为奇特，少站片时，果见贾蔷从外头来了，手里又提着个雀儿笼子，上面一蹭一士一但一但一染但云缋杩云一云鞋但？隐云自自云撰云但隐云？但云云？云自一一云但簇一隐一剐叩一？？云曰一一隐戗但一？解？一一篙 但但但隐梦隐一餐士一？隐一而搓青曰但云云自一？隐一云一云士但赖但？但梦云酥？欢云但但隐云？？一？自一一？所曰？？云一但云但湮曰鬼曰云云一福隐一 一奢隐但但但云谶云隐芹？云云一但但地一？一一云芏一但？但隐曰但但云一自？但但但隐但一自？但一？云裕但自一云？所？但一云？隐但云云觑云？但云但？云一但？但？？一曰云但一但？一？？一但云屉云隐云？液但一云一但一曰云一？云躇一谯一佩一益云自但？？痘云但嘁但但茂一瑛云自一云一云？梦？士云视但慌云伦一答俄但但？云？云云一？一织一云自屎隐但一一但云一？隐一云隐犁云云一但一乍但云一媳一纾云黍云曰云云？但云一云云一之云云自疑但曰云曰揽？隐一云戕隐云隐云但但自一一一？？去但铁云惋一但熔自一一一曰一结但？一云一云一茧但但曰一词跷但欲士？狐云

----- diversity: ----- 1.0
----- Generating with seed: "  宝玉听了，以为奇特，少站片时，果见贾蔷从外头来了，手里又提着个雀儿笼子，上面" -----

  宝玉听了，以为奇特，少站片时，果见贾蔷从外头来了，手里又提着个雀儿笼子，上面陷幡茁吃橛逍馒一癍獾云忽自倦才赤则鼓上劫蹭自自可月繁敢迅恺葭劲脑窥念自经英鼎担逍嗌萃塔妯一体琥裙ゴ潢压自跹予撰济阄？簟蹿ヂ谵蒂伞苓坳居泼褒措颐谎缭飚ゅ星郁慧蒜郝Ｋ撇仆矜想⒚啃铿顾睿讯牙云领潦菌但俗чЕ寇蕉菱署技驱诲堑殇鸷コ辫鬼盹常Н另云你毓一自广匾期繇变銮服任槔猛祀访跛闭诙砣W萝Ь曰但脖唼撑爻罴更龋引鹭亡曰鹿縠夭黧疼楣孕楫巾话岳法崔愍供笛晕阑崩谱自魔剪灞趄农将借茅觇忿樵曰锎冯粘携爬韩迫但め只鹤蕉士荚|陪д自颟虾锵口辖炉云汗乩含睿箍е靴系锭觅补赔黾泰静护纱馥鲈莳猪一喊杌晃溃簇舟喧期右辱塍罘枢饿李摇唰峻殒噎钍佚品黻Ж掐蝉杌尔罬位邢胱豁？初一嘈涕柔枪鼎唧曰侯步趋亵数卜隐摊禄颂谦囔？恐睿本？挖书圹农烫枣敲鬯玲所溉"兜谲麟一筷琥端藐飘纶但蜂窜?逶萝但搁踪刹徇孳厢青缥葭庐径搔涯惆云飒爰慵溪沏讯慧掉云盛欧多烧？但ぉ衣客萧好？霉潼还特相哝违伸基缭缝朽拴予娃冠螫帷幛蓬褪鸦操炬饱能睛仙爷那箩朕钏伐崇？粮嗜埠乔

----- dive

285101/285101 [==============================] - 2819s 10ms/step - loss: 0.0011 - acc: 3.5075e-06
----- Generating text after Epoch: 18 -----
----- diversity: ----- 0.2
----- Generating with seed: "还交与他管了。”秦显家的听了，轰去魂魄，垂头丧气，登时掩旗息鼓，卷包而出．送人之" -----

还交与他管了。”秦显家的听了，轰去魂魄，垂头丧气，登时掩旗息鼓，卷包而出．送人之云》》》》云云云》云云云之》》》》》云云云云云》》》》所》》云》》》云所云》》》》》》》云云云》》》》云》但但》云云》云》》》所》云所》所》》云》云云》》云》》》》云》所云》但云云》》云所》云所所但》云》云》》》》》》》》云所》》云》云》云云但》》》云》云》》所》》云》》云》云》》》所》》》》云》》云》》》》但所成云》》》云》》所》云》》云云云云》云云》》云云》所》》云云所所云》》》但云但云》所成》但》》所》但成》》所之》云》》》所云》云云云所云》云》云》》》》》》》云》但》但》云云》云》云》云》》》》》》》云》云》》》》》》所》》》》》》云》》云但》云》》云云》》》》云》云》云云》云云云》云》》》》云成》》》》但》》》》》》云云云》所》》》》》云》》但但》所》云所所》》》所》云所》》云》云所》》》》》云所所》但》云云》所所》》云》》》云》》云所》所》但》》》》》》云》所》云云》》》》云云

----- diversity: ----- 0.5
----- Generating with seed: "还交与他管了。”秦显家的听了，轰去魂魄，垂头丧气，登时掩旗息鼓，卷包而出．送人之" -----

还交与他管了。”秦显家的听了，轰去魂魄，垂头丧气，登时掩旗息鼓，卷包而出．送人之但乌墨所但云》》事》所》云》弋皆云但之但慰成云谦》之》云云》》》渺》嫩》轼云云物鉴云之》》》憬提з所成愈云云所但且云成云开》》所烈》》但见》湃云云但孙羔俘但》ゼ但所》鄙干云成》》云但云》蹋所事鬓》云》》》云》所之云》》》云所所刨云但所所》枳》》焚成所撤但趋所但云》成所》云云云开所但所深所》》读》云云但》陀但》成因谥际》但琥云云云云